In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio

In [2]:
possibleModels = [item["model"] for item in curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)]
allModels = []
tickersSeen = []
for i in range(len(possibleModels)):
    model = possibleModels[i]
    if model.numberOfPredictors() > 4:
        print(model.targetTicker, model.getHash(), model.numberOfPredictors())
        allModels.append(model)
        if model.targetTicker not in tickersSeen:
            tickersSeen.append(model.targetTicker)

GDX f6543a2b901c10304f3883ef2fcc3d379e26337b2da0a30aafcbd2a5 5
KRE d9952eb623eff7801cd891ee38aed3cb31c8472ec8ee8522cdc42308 5
XOP 8517ad01fb7a6f4db74903f3029c8518263592bc3d833026cb1e3d6f 5
VNQ 1c8e43f5b60ab5dc57013cb2511cc46f23363e27dd77ebc7c76a5c65 6
IVE 326b240a831a1d2a1cf0b1cb333aa57602243e7821173b7892d900d0 6
VGK 1c661e71ead587b1df19a85028f376754e18bbc50e9551ca8f83f9e6 8
KBE 5da44ed8a1de56102495514d68c8f046e095075fd021de520b54efb4 6
VNQ fc0a352fbc61beea08025e49b5cc389a563daf3d1d62a3af9f8acb3b 6
XOP 519596acca2f56d76c290c69aaea62901b9f78925d786db3a7a5cd5c 6
EEM db20ba01634fcbc39e74523a0bb7a59858942f81d300754ef37dd3a9 6
XOP 703cb37e7209960a2587ec99abd037b3abe31b89c896175cdbb45c04 6
XME 1ce2dd6a918ad3b7da8b593c66bd18ba70b42bfd0a1b8488b0f1a3bd 6
XME 6059471561fe0357e249f7c3531d96c595c6bbfbee1d4226f4d593be 6
XOP 5ab622e877acfee8539c3deb00514b5317c420bcfee4e55a10446f07 6
SMH 32619fd325a2a232f72fc6d3e0a981d7174e978fb0a38f42dbf13473 5
SLV 004a5100d928327a3d81e1070aa29015181eb0e3c53267d45a9

In [3]:
len(allModels)

91

In [4]:
tickersSeen

['GDX',
 'KRE',
 'XOP',
 'VNQ',
 'IVE',
 'VGK',
 'KBE',
 'EEM',
 'XME',
 'SMH',
 'SLV',
 'XLF',
 'XBI',
 'VFH',
 'EWZ',
 'SOXX']

In [5]:
import random
factorToTrade = tickersSeen[random.randint(0, len(tickersSeen) - 1)]
factorToTrade

'EWZ'

In [6]:
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


['GDX', 'KRE', 'XOP', 'VNQ', 'IVE', 'VGK', 'KBE', 'EEM', 'XME', 'SMH', 'SLV', 'XLF', 'XBI', 'VFH', 'EWZ', 'SOXX']
GDX
KRE
XME
XOP
KBE
VNQ
VGK
EEM
IVE
SLV
XBI
SMH
VFH
SOXX
EWZ
XLF
ATTEMPTING PULL f6543a2b901c10304f3883ef2fcc3d379e26337b2da0a30aafcbd2a5
((('IVW', 8, None, None, 2), 'GDX', 22, 5, 1.0, 0.05, 10), (((('VB', 32, None, None, 1), 'GDX', 10, 5, 1.0, 0.1, 10), (('IWB', 15, 11, 9, 5), 'GDX', 22, 5, 1.5, 0.1, 10), 5, 'OR'), ((('SMH', 41, 3, None, 1), 'GDX', 5, 5, 0.5, 0.01, 10), (('XLK', 23, 5, None, 1), 'GDX', 10, 5, 1.5, 0.05, 10), 5, 'OR'), 5, 'OR'), 5, 'OR')
ATTEMPTING PULL d9952eb623eff7801cd891ee38aed3cb31c8472ec8ee8522cdc42308
(((('VGK', 4, 12, 22, 2), 'KRE', 10, 5, 0.5, 0.05, 10), (('IVE', 2, None, None, 5), 'KRE', 22, 5, 1.5, 0.05, 10), 5, 'AND'), ((('XBI', 11, None, None, 1), 'KRE', 10, 5, 0.7, 0.01, 10), ((('EWW', 13, 12, None, 1), 'KRE', 22, 5, 1.0, 0.1, 10), (('IWB', 36, 1, None, 4), 'KRE', 10, 5, 1.0, 0.1, 10), 5, 'OR'), 5, 'AND'), 5, 'OR')
ATTEMPTING PULL 8517ad01fb

In [156]:
cleanedReturns = modelReturns.dropna()
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.dropna()
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item

In [157]:
def historicalWeightsToTickerAllocations(historicalWeights, algorithmPredictions, modelsInPortfolio):
    aggregatePredictions = algorithmPredictions.dropna()
    allocationsToStore = []
    historicalAllocations = None
    scaledHistoricalAllocations = None
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.targetTicker not in netPosition:
                netPosition[model.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.targetTicker] += weights[model.getHash()] * aggregatePredictions.loc[historicalWeights.index[i]][model.getHash()]
        thisDf = pd.DataFrame([netPosition], index=[historicalWeights.index[i]])
        if historicalAllocations is None:
            historicalAllocations = thisDf
        else:
            historicalAllocations = pd.concat([historicalAllocations, thisDf])
        
        totalCapitalUsed = sum([abs(netPosition[ticker]) for ticker in netPosition])
        scaledNetPosition = {}
        for ticker in netPosition:
            scaledNetPosition[ticker] = netPosition[ticker] * 1.0/totalCapitalUsed
        
        thisDf = pd.DataFrame([scaledNetPosition], index=[historicalWeights.index[i]])
        if scaledHistoricalAllocations is None:
            scaledHistoricalAllocations = thisDf
        else:
            scaledHistoricalAllocations = pd.concat([scaledHistoricalAllocations, thisDf])
    
    return historicalAllocations, scaledHistoricalAllocations
            
            

In [158]:
import empyrical
def getLimitedDataForPortfolio(historicalWeights, historicalPredictions, modelsUsed, factorToTrade, joinedData):
    
    normalTickerAllocationsTable, scaledTickerAllocationsTable = historicalWeightsToTickerAllocations(historicalWeights, historicalPredictions, modelsUsed)
    
    tickerAllocationsTable = scaledTickerAllocationsTable
    rawTickerPerformance = portfolioGeneration.calculatePerformanceForTable(tickerAllocationsTable, tickerAllocationsTable.columns, joinedData)

    rawAlgoPerformance = pd.DataFrame(rawTickerPerformance.apply(lambda x:sum(x), axis=1), columns=["Algo Return Without Commissions"])

    tickerPerformance, algoPerformance, algoTransactionCost =  portfolioGeneration.calculatePerformanceForAllocations(tickerAllocationsTable, joinedData)

    benchmark = factorToTrade
    factorReturn = dataAck.getDailyFactorReturn(benchmark, joinedData)
    factorReturn.columns = ["Factor Return (" + benchmark + ")"]
    algoPerformance.columns = ["Algo Return"]

    alpha, beta = empyrical.alpha_beta(algoPerformance, factorReturn)
    sharpe_difference = empyrical.sharpe_ratio(algoPerformance) - empyrical.sharpe_ratio(factorReturn)
    annualizedReturn = empyrical.annual_return(algoPerformance)[0]
    annualizedVolatility = empyrical.annual_volatility(algoPerformance)
    
    ##AUTOMATICALLY TAKES SLIPPAGE INTO ACCOUNT
    return {
        "benchmark":factorToTrade,
        "alpha":alpha,
        "beta":beta,
        "sharpe difference":sharpe_difference,
        "annualizedReturn":annualizedReturn,
        "annualizedVolatility":annualizedVolatility,
        "sharpe":empyrical.sharpe_ratio(algoPerformance),
        "free return":annualizedReturn - annualizedVolatility
    }
    


In [159]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(10, len(algoColumns)), replace= False)


In [160]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
        historicalWeights = pd.concat([historicalWeights, thisWeights])
        i += 1
    return hrpReturns, historicalWeights

In [ ]:
def storeDiscoveredPortfolio(models, benchmark, IS_DATA, OOS_DATA):
    description = "AUTO GENERATED"
    portfolioType = "HRP FULL"
    seenTickers = []
    
    allHashes = []
    for model in models:
        allHashes.append(model.getHash())
        if model.targetTicker not in seenTickers:
            seenTickers.append(model.targetTicker)
        
    ##SORT SO ENSURE SAME PORTFOLIO NOT CREATED TWICE
    allHashes = sorted(allHashes)
    
    portfolioString = str(allHashes) + benchmark + description + portfolioType
    portfolioHash = hashlib.sha224(portfolioString.encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.discoveredPortfolioModels, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark,
                "portfolioType":portfolioType,
                "startedTrading":curveTreeDB.getToday()
            }
            
            for k in IS_DATA:
                toUpload["IS_"+ k] = IS_DATA[k]
            
            for k in OOS_DATA:
                toUpload["OOS_"+ k] = OOS_DATA[k]
                
            for ticker in seenTickers:
                toUpload[ticker] = True
            
            toUpload["TICKERS TRADED"] = len(seenTickers)
            
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.discoveredPortfolios, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            return portfolioHash
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [161]:
def performPortfolioPerformanceEstimation(historicalPredictions, historicalReturns, factorToTrade, hashToModel, joinedData):
    returnWindows = [(None, historicalReturns[:400]), (400, historicalReturns)]
    historicalWeights = None
    for selectedReturns in returnWindows:
        startIndex = selectedReturns[0]
        returnWindow = selectedReturns[1]
        hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                126, startIndex=startIndex, maxWindowSize=False)
        print("COMPUTED HISTORICAL WEIGHTS")
        if historicalWeights is None:
            historicalWeights = weightsSeen
        else:
            historicalWeights = pd.concat([historicalWeights, weightsSeen])
        
        modelsUsed = []

        tickersSeen = {}

        for modelHash in historicalPredictions.columns:
            thisModel = hashToModel[modelHash]
            modelsUsed.append(thisModel)
        if startIndex is None:
            scaledStats = getLimitedDataForPortfolio(historicalWeights,\
                                    historicalPredictions, modelsUsed, factorToTrade, joinedData)
            print(scaledStats)
            if scaledStats["sharpe difference"] < 0.0 or scaledStats["annualizedReturn"] < scaledStats["annualizedVolatility"]:
                return None, None
    trainStats = getLimitedDataForPortfolio(historicalWeights[:-252], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    testStats = getLimitedDataForPortfolio(historicalWeights[-252:], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    
    if trainStats["sharpe difference"] > 0.0 and trainStats["annualizedReturn"] > trainStats["annualizedVolatility"]:
        print(trainStats, testStats)
        storeDiscoveredPortfolio(modelsUsed, trainStats, testStats)
    

In [163]:
## MP RUN            
def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        print(len(selectedAlgorithms))
        factorToTrade = hashToModel[selectedAlgorithms[random.randint(0, len(selectedAlgorithms) - 1)]].targetTicker
        print(factorToTrade)
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, hashToModel, joinedData))
        p.start()
        runningP.append(p)

# In[ ]:


In [164]:
print("STARTING GENERATION")

##REMOVE BREAK TO DO FULL AUTO
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=0)

STARTING GENERATION
21
XHB
24
ITB
COMPUTED HISTORICAL WEIGHTS
{'benchmark': 'XHB', 'alpha': 0.0648249539358521, 'beta': 0.018388947856885399, 'sharpe difference': 2.5263597049864885, 'annualizedReturn': 0.067285208111367734, 'annualizedVolatility': 0.02438993405029084, 'sharpe': 2.6823617736370462} {'benchmark': 'XHB', 'alpha': 0.4463321282277181, 'beta': 0.087266889674225051, 'sharpe difference': 2.1676995768951164, 'annualizedReturn': 0.53774648279405679, 'annualizedVolatility': 0.19329867710974166, 'sharpe': 2.323701645545674}
COMPUTED HISTORICAL WEIGHTS
train ({'benchmark': 'XHB', 'alpha': 0.06513660139910891, 'beta': 0.016070976544025208, 'sharpe difference': 2.4665639303882205, 'annualizedReturn': 0.06769437300722414, 'annualizedVolatility': 0.025098765730737244, 'sharpe': 2.6225659990387782}, {'benchmark': 'XHB', 'alpha': 0.4366631000089456, 'beta': 0.062340481134984096, 'sharpe difference': 2.0303442154046687, 'annualizedReturn': 0.52049811301836613, 'annualizedVolatility': 0.2

Process ForkProcess-377:
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1325, in __getitem__
    return self._getitem_tuple(key)
Traceback (most recent call last):
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-161-44b81b9db6d6>", line 7, in performPortfolioPerformanceEstimation
    hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,                126, startIndex=startIndex, maxWindowSize=False)
  File "<ipython-input-160-bdd8563466f0>", line 17, in produceHRPPredictions
    weights = hrp.getHRP(cov, corr)
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/hrpPortfolioOpt.py", line 98, in getHRP
    hrp = getRecBipart(cov, sortIx)
  File "/Users/patrickogrady/Documents/CODE/wal

KeyboardInterrupt: 